In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
from datetime import date, datetime
fname = 'https://www.worldometers.info/coronavirus/'
req = Request(fname, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req)
page_soup = soup(webpage, "html.parser")

In [2]:
today = datetime.now()
now_str = "%s %d, %d at %d:%s" % (date.today().strftime("%b"), today.day, today.year, today.hour, '0'+str(today.minute) if today.minute < 10 else str(today.minute))
containers = page_soup.findAll("div", {"class": "maincounter-number"})
print("As of %s UTC, there have been %s total COVID-19 cases." % (now_str, containers[0].findAll("span")[0].text.replace(' ', '')))

As of Sep 8, 2023 at 13:34 UTC, there have been 695,063,811 total COVID-19 cases.


In [3]:
#-----General------#
import numpy as np
import pandas as pd
import os
import sys
import math
import random

#-----Plotting-----#
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
py.init_notebook_mode(connected=True)
import seaborn as sns
from pandas_profiling import ProfileReport

#-----Utility-----#
import itertools
import warnings
warnings.filterwarnings("ignore")
import re
import gc
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
from datetime import date, datetime

LOOK_AT = 5 # Controls how many bars the user can see in the bar graph
AT_LEAST = 50 # Controls what rank a country must be in terms of total cases to be shown on the bar graph

/opt/conda/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning:

numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.

/opt/conda/lib/python3.10/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

/tmp/ipykernel_20/3028595598.py:16: DeprecationWarning:

`import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.



<h1> Web Scraping Foundation </h1>

In [4]:
fname = 'https://www.worldometers.info/coronavirus/#countries'
req = Request(fname, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req)
page_soup = soup(webpage, "html.parser")
today = datetime.now()
today_str = "%s %d, %d" % (date.today().strftime("%b"), today.day, today.year)
yesterday_str = "%s %d, %d" % (date.today().strftime("%b"), today.day-1, today.year)
clean = True

In [5]:
print("This version of the notebook is being run on %s." % today_str)

This version of the notebook is being run on Sep 8, 2023.


In [6]:
table = page_soup.findAll("table", {"id": "main_table_countries_yesterday"})
containers = table[0].findAll("tr", {"style": ""})
del containers[0]

all_data = []
for country in containers:
    country_data = []
    country_container = country.findAll("td")
    if country_container[1].text == 'China':
        continue
    for i in range(1, len(country_container)):
        final_feature = country_container[i].text
        if clean:
            if i != 1 and i != len(country_container)-1:
                final_feature = final_feature.replace(',', '')
                if final_feature.find('+') != -1:
                    final_feature = final_feature.replace('+', '')
                    final_feature = float(final_feature)
                elif final_feature.find('-') != -1:
                    final_feature = final_feature.replace('-', '')
                    final_feature = float(final_feature)*-1
        if final_feature == 'N/A':
            final_feature = 0
        elif final_feature == '' or final_feature == ' ':
            final_feature = -1 #None
        country_data.append(final_feature)
    all_data.append(country_data)

In [7]:
df = pd.DataFrame(all_data)
df = df.drop([i for i in range(15, len(all_data[0]))], axis=1) # Get rid of unnecessary data

On the <a href="https://www.worldometers.info/coronavirus/"> worldometers website</a>, the category "New Recovered" doesn't appear; however, based on the numbers, we can interpolate a certain column of data to be that.

In [8]:
column_labels = ["Country", "Total Cases", "New Cases", "Total Deaths", "New Deaths", "Total Recovered", "New Recovered", "Active Cases", "Serious/Critical",
                "Tot Cases/1M", "Deaths/1M", "Total Tests", "Tests/1M", "Population", "Continent"]
df.columns = column_labels

<h1> What Countries are not present in the Analysis? </h1>

For some reason, there are some countries that are not included when scraping the webpage.

In [9]:
country_labels = page_soup.findAll("a", {"class": "mt_a"})
c_label = []
for country in country_labels:
    c_label.append(country.text)
c_label = set(c_label)

not_counted = []
sorted_countries = set(df['Country']) #Increase computational speed
for country in c_label:
    if country not in sorted_countries:
        not_counted.append(country)

In [10]:
print(not_counted)

['Sao Tome and Principe', 'Greece', 'Netherlands', 'Cook Islands', 'China', 'Serbia', 'Falkland Islands', 'Taiwan', 'Mongolia', 'Cambodia', 'Sudan', 'France', 'Saint Kitts and Nevis', 'Ukraine', 'Western Sahara', 'Peru', 'DPRK', 'Ecuador', 'Vatican City', 'Cyprus', 'Eritrea', 'Syria']


<h1> Final Processing </h1>

Here, we will convert all the numerical data into np.int64 data type, and add some other features that may be particularly useful.

In [11]:
for label in df.columns:
    if label != 'Country' and label != 'Continent':
        df[label] = pd.to_numeric(df[label])

In [12]:
df['%Inc Cases'] = df['New Cases']/df['Total Cases']*100
df['%Inc Deaths'] = df['New Deaths']/df['Total Deaths']*100
df['%Inc Recovered'] = df['New Recovered']/df['Total Recovered']*100

In [13]:
pd.options.display.max_rows = None
df

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Tot Cases/1M,Deaths/1M,Total Tests,Tests/1M,Population,Continent,%Inc Cases,%Inc Deaths,%Inc Recovered
0,World,695061924,10669.0,6913771,30.0,666853737,62408.0,21294416,37769,89170.0,887.0,-1,-1,-1,All,0.001535,0.000434,0.009359
1,USA,108280870,9389.0,1174558,22.0,106067796,17446.0,1038516,1350,323414.0,3508.0,1186093362,3542636,334805269,North America,0.008671,0.001873,0.016448
2,India,44997537,-1.0,531930,-1.0,44465019,-1.0,588,0,31990.0,378.0,930797975,661721,1406631776,Asia,-0.000002,-0.000188,-0.000002
3,Germany,38428685,-1.0,174352,-1.0,38240600,-1.0,13733,0,458119.0,2078.0,122332384,1458359,83883596,Europe,-0.000003,-0.000574,-0.000003
4,Brazil,37783855,-1.0,705313,-1.0,36249161,-1.0,829381,0,175450.0,3275.0,63776166,296146,215353593,South America,-0.000003,-0.000142,-0.000003
5,S. Korea,34571873,-1.0,35934,-1.0,34129296,38989.0,406643,231,673523.0,700.0,15804065,307892,51329899,Asia,-0.000003,-0.002783,0.114239
6,Japan,33803572,-1.0,74694,-1.0,0,0.0,0,83,269169.0,595.0,100414883,799578,125584838,Asia,-0.000003,-0.001339,NaN
7,Italy,25955703,-1.0,191276,-1.0,25639992,-1.0,124435,19,430709.0,3174.0,274852965,4560908,60262770,Europe,-0.000004,-0.000523,-0.000004
8,UK,24688073,-1.0,228924,-1.0,24438058,1043.0,21091,0,360421.0,3342.0,522526476,7628357,68497907,Europe,-0.000004,-0.000437,0.004268
9,Russia,22995765,-1.0,399999,-1.0,22446519,-1.0,149247,0,157715.0,2743.0,273400000,1875095,145805947,Europe,-0.000004,-0.000250,-0.000004


<h1> Export </h1>

Feel free to use this data for your own purposes/visualizations. If you don't want to fork the notebook, you can download the csv file in the output section of this notebook.

In [14]:
EXPORT = True
today = datetime.now()
if EXPORT:
    today = date.today()
    df.to_csv(f'covid_stats_{today.year}_{today.month}_{today.day-1}')
    print("Dataset is %.2f MB" % (df.memory_usage(deep=True).sum()/1000000))

Dataset is 0.05 MB


<h1 class="alert alert-block alert-info" style="text-align:center; font-size:24px" id="general">General Visualizations<a class="anchor-link" href="https://www.kaggle.com/ironicninja/covid-19-every-day/notebook#general">¶</a></h1>

In [15]:
cases_ser = df[["Total Recovered", "Active Cases", "Total Deaths"]].loc[0]
cases_df = pd.DataFrame(cases_ser).reset_index()
cases_df.columns = ['Type', 'Total']
cases_df['Percentage'] = np.round(100*cases_df['Total']/np.sum(cases_df['Total']), 2)
cases_df['Virus'] = ['COVID-19' for i in range(len(cases_df))]

fig = px.bar(cases_df, x='Virus', y='Percentage', color='Type', hover_data=['Total'])
fig.update_layout(title={'text': f"Total Number of Cases, Recoveries, and Deaths on {yesterday_str}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20}}, yaxis_title="Percentage", xaxis_title="")
fig.show()

In [16]:
new_ser = df[["New Cases", "New Recovered", "New Deaths"]].loc[0]
new_df = pd.DataFrame(new_ser).reset_index()
new_df.columns = ['Type', 'Total']
new_df['Percentage'] = np.round(100*new_df['Total']/np.sum(new_df['Total']), 2)
new_df['Virus'] = ['COVID-19' for i in range(len(new_df))]

fig = px.bar(new_df, x='Virus', y='Percentage', color='Type', hover_data=['Total'])
fig.update_layout(title={'text': f"New Cases, Recoveries, and Deaths on {yesterday_str}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20}}, yaxis_title="Percentage", xaxis_title="")
fig.show()

In [17]:
pinc_ser = np.round(df[["%Inc Cases", "%Inc Recovered", "%Inc Deaths"]].loc[0], 2)
pinc_df = pd.DataFrame(pinc_ser)
pinc_df.columns = ["Percentage"]

fig = go.Figure()
fig.add_trace(go.Bar(x=pinc_df.index, y=pinc_df['Percentage'], marker_color=["yellow", "green", "red"]))
fig.update_layout(title={'text': f"New Cases, Recoveries, and Deaths on {yesterday_str}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20}}, yaxis_title="Percentage", xaxis_title="")
fig.show()

<h1 class="alert alert-block alert-info" style="text-align:center; font-size:24px" id="continent">By Continent<a class="anchor-link" href="https://www.kaggle.com/ironicninja/covid-19-every-day/notebook#continent">¶</a></h1>

In [18]:
continent_df = df.groupby('Continent').sum().drop('All')
continent_df = continent_df.reset_index()
continent_df

,Continent,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Tot Cases/1M,Deaths/1M,Total Tests,Tests/1M,Population,%Inc Cases,%Inc Deaths,%Inc Recovered
0,Africa,South AfricaMoroccoTunisiaEgyptLibyaEthiopiaRé...,12756412,-54.0,253598,-54.0,10894608,-50.0,392560,516,2462778.0,18053.0,110408866,10773967,1356220640,-0.205484,86.191168,-50.159160
1,Asia,IndiaS. KoreaJapanTurkeyVietnamIranIndonesiaMa...,203331708,-5.0,1515503,-43.0,148402033,38970.0,2337330,1018,6568898.0,32117.0,2190197204,82560139,3171871196,-0.029784,-8.651036,0.058369
2,Australia/Oceania,AustraliaNew ZealandNew CaledoniaFrench Polyne...,14622906,-19.0,30244,-19.0,14430496,-16.0,33303,37,4439276.0,9201.0,91647660,20126840,43452837,-1.519574,-54.673447,199.498797
3,Europe,GermanyItalyUKRussiaSpainPolandAustriaPortugal...,186943482,614.0,1712248,-33.0,183280667,5252.0,1669083,672,16704575.0,115219.0,2389888141,197173189,602585298,-0.001894,-10.558956,0.088707
4,North America,USAMexicoCanadaGuatemalaCosta RicaCubaPanamaDo...,127997932,9629.0,1644432,-12.0,123464897,17673.0,1982009,1703,8749185.0,58562.0,1324457203,99333266,598087045,-0.242127,-110.496795,-0.336498
5,South America,BrazilArgentinaColombiaChileBoliviaUruguayPara...,63384720,309.0,1102371,-11.0,60435913,483.0,971634,986,1863811.0,24561.0,202891194,10241798,385893335,0.001924,-0.428896,-0.001275


In [19]:
cases_vis_list = ['Total Cases', 'Active Cases', 'New Cases', 'Serious/Critical', 'Tot Cases/1M']
deaths_vis_list = ['Total Deaths', 'New Deaths', 'Deaths/1M']
recovered_vis_list = ['Total Recovered', 'New Recovered']
tests_vis_list = ['Total Tests', 'Tests/1M']
essentials = [['Total Cases', 'Active Cases', 'New Cases'], ['Total Deaths', 'New Deaths'], ['Total Recovered', 'New Recovered']]

In [20]:
def continent_visualization(vis_list):
    for label in vis_list:
        c_df = continent_df[['Continent', label]]
        c_df['Percentage'] = np.round(100*c_df[label]/np.sum(c_df[label]), 2)
        c_df['Virus'] = ['COVID-19' for i in range(len(c_df))]
        
        fig = px.bar(c_df, x='Virus', y='Percentage', color='Continent', hover_data=[label])
        fig.update_layout(title={'text': f"{label} at the end of {yesterday_str}", 'x': 0.5,
                                 'xanchor': 'center', 'font': {'size': 20}}, yaxis_title="Percentage", xaxis_title="")
        fig.show()
        gc.collect()

In [21]:
continent_visualization(cases_vis_list)

In [22]:
continent_visualization(deaths_vis_list)

In [23]:
continent_visualization(tests_vis_list)

In [24]:
def continent_visualization2(index, label, log_scale=False):
    c_df = continent_df[['Continent'] + essentials[index]]
    
    fig = px.bar(c_df, x="Continent", y=essentials[index], log_y=log_scale)
    log_str = "- log scale" if log_scale else ""
    fig.update_layout(title={'text': f"{label} at the end of {yesterday_str} {log_str}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20}}, yaxis_title=label, xaxis_title="")
    
    fig.show()
    gc.collect()

In [25]:
continent_visualization2(0, "Total Cases", log_scale=True)

In [26]:
continent_visualization2(1, "Deaths", log_scale=True)

In [27]:
continent_visualization2(2, "Recoveries", log_scale=True)

<h1 class="alert alert-block alert-info" style="text-align:center; font-size:24px" id="countries">By Countries<a class="anchor-link" href="https://www.kaggle.com/ironicninja/covid-19-every-day/notebook#countries">¶</a></h1>

In [28]:
df = df.drop([len(df)-1])
country_df = df.drop([0])

In [29]:
country_l = country_df.columns[1:14]

fig = go.Figure()
c = 0
for i in country_df.index:
    if c < LOOK_AT:
        fig.add_trace(go.Bar(name=country_df['Country'][i], x=country_l, y=country_df.loc[i][1:14]))
    else:
        break
    c += 1
    
fig.update_layout(title={'text': f'{LOOK_AT} Countries with Most COVID Cases on %s' % yesterday_str, 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20}}, yaxis_title="Percentage", yaxis_type="log", xaxis_tickangle=-90)
fig.show()

In [30]:
inc_l = country_df.columns[15:]
inc_df = country_df.sort_values("%Inc Cases", ascending=False)
fig = go.Figure()
c = 0
for i in inc_df.index:
    if i > AT_LEAST:
        continue
    if c < LOOK_AT:
        fig.add_trace(go.Bar(name=country_df['Country'][i], x=inc_l, y=inc_df.loc[i][15:]))
    else:
        break
    c += 1
    
fig.update_layout(title={'text': f'{LOOK_AT} Countries with Highest Increase in COVID Cases on %s' % yesterday_str, 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20}}, yaxis_title="Percentage", xaxis_tickangle=0)
fig.show()

In [31]:
country_labels = country_df.columns[1:14]

def country_visualization(continent):
    buttons_list = []
    base_list = [False for i in range(len(country_df))]
    c = 0
    for i in country_df.index:
        if country_df.loc[i]['Continent'] != continent:
            continue
        tmp_list = base_list.copy()
        tmp_list[c] = True
        c += 1
        buttons_list.append(dict(
                    args=[{"visible": tmp_list}],
                    label=country_df.loc[i]['Country'],
                    method="update"
                ))


    fig = go.Figure()
    c = 0
    for i in country_df.index:
        if country_df.loc[i]['Continent'] != continent:
            continue
        fig.add_trace(go.Bar(name=country_df.loc[i]['Country'], x=country_labels, y=country_df.loc[i][1:14], visible=False if c != 0 else True))
        c += 1

    fig.update_layout(
        updatemenus=[
            dict(
                buttons=buttons_list,
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.1,
                xanchor="left",
                y=1.1,
                yanchor="top"
            ),
        ]
    )

    fig.update_layout(title={'text': '%s COVID-19 Cases Search on %s' % (continent, yesterday_str), 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20}}, yaxis_type="log", xaxis_tickangle=-90)
    fig.show()

<h1 id="country-search">COVID-19 Cases Search</h1>

In [32]:
country_visualization('Africa')

In [33]:
country_visualization('Asia')

In [34]:
country_visualization('Australia/Oceania')

In [35]:
country_visualization('Europe')

In [36]:
country_visualization('North America')

In [37]:
country_visualization('South America')

In [38]:
bar_list = []
for i in country_df.index:
    bar_list.append(go.Bar(name=country_df['Country'][i], y=[country_df['Total Cases'][i]]))
    
fig = go.Figure(data=bar_list)
fig.update_layout(title={'text': 'Stacked Bar Chart of All Countries COVID-19 Cases on %s' % yesterday_str, 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20}}, barmode='stack', height=1200)
fig.show()

In [39]:
bar = go.Bar(x=country_df['Country'], y=country_df['Total Cases'], marker=dict(color=df['Total Cases'], colorscale='Reds', showscale=True))
fig = go.Figure(data=[bar])
fig.update_layout(title={'text': 'Number of COVID Cases by Country on %s, log scale' % yesterday_str, 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20}}, yaxis_type="log", xaxis_tickangle=-90)

<h1 id="choropleth"> Choropleth Visualizations </h1>

In [40]:
country_abbrev = {
'Afghanistan': 'AFG',
 'Albania': 'ALB',
 'Algeria': 'DZA',
 'American Samoa': 'ASM',
 'Andorra': 'AND',
 'Angola': 'AGO',
 'Anguilla': 'AIA',
 'Antigua and Barbuda': 'ATG',
 'Argentina': 'ARG',
 'Armenia': 'ARM',
 'Aruba': 'ABW',
 'Australia': 'AUS',
 'Austria': 'AUT',
 'Azerbaijan': 'AZE',
 'Bahamas, The': 'BHM',
 'Bahrain': 'BHR',
 'Bangladesh': 'BGD',
 'Barbados': 'BRB',
 'Belarus': 'BLR',
 'Belgium': 'BEL',
 'Belize': 'BLZ',
 'Benin': 'BEN',
 'Bermuda': 'BMU',
 'Bhutan': 'BTN',
 'Bolivia': 'BOL',
 'Bosnia and Herzegovina': 'BIH',
 'Botswana': 'BWA',
 'Brazil': 'BRA',
 'British Virgin Islands': 'VGB',
 'Brunei': 'BRN',
 'Bulgaria': 'BGR',
 'Burkina Faso': 'BFA',
 'Burma': 'MMR',
 'Burundi': 'BDI',
 'Cabo Verde': 'CPV',
 'Cambodia': 'KHM',
 'Cameroon': 'CMR',
 'Canada': 'CAN',
 'Cayman Islands': 'CYM',
 'Central African Republic': 'CAF',
 'Chad': 'TCD',
 'Chile': 'CHL',
 'China': 'CHN',
 'Colombia': 'COL',
 'Comoros': 'COM',
 'Congo, Democratic Republic of the': 'COD',
 'Congo, Republic of the': 'COG',
 'Cook Islands': 'COK',
 'Costa Rica': 'CRI',
 "Cote d'Ivoire": 'CIV',
 'Croatia': 'HRV',
 'Cuba': 'CUB',
 'Curacao': 'CUW',
 'Cyprus': 'CYP',
 'Czech Republic': 'CZE',
    'Czechia': 'CZE',
 'Denmark': 'DNK',
 'Djibouti': 'DJI',
 'Dominica': 'DMA',
 'Dominican Republic': 'DOM',
 'Ecuador': 'ECU',
 'Egypt': 'EGY',
 'El Salvador': 'SLV',
 'Equatorial Guinea': 'GNQ',
 'Eritrea': 'ERI',
 'Estonia': 'EST',
 'Ethiopia': 'ETH',
 'Falkland Islands (Islas Malvinas)': 'FLK',
 'Faroe Islands': 'FRO',
 'Fiji': 'FJI',
 'Finland': 'FIN',
 'France': 'FRA',
 'French Polynesia': 'PYF',
 'Gabon': 'GAB',
 'Gambia, The': 'GMB',
 'Georgia': 'GEO',
 'Germany': 'DEU',
 'Ghana': 'GHA',
 'Gibraltar': 'GIB',
 'Greece': 'GRC',
 'Greenland': 'GRL',
 'Grenada': 'GRD',
 'Guam': 'GUM',
 'Guatemala': 'GTM',
 'Guernsey': 'GGY',
 'Guinea-Bissau': 'GNB',
 'Guinea': 'GIN',
 'Guyana': 'GUY',
 'Haiti': 'HTI',
 'Honduras': 'HND',
 'Hong Kong': 'HKG',
 'Hungary': 'HUN',
 'Iceland': 'ISL',
 'India': 'IND',
 'Indonesia': 'IDN',
 'Iran': 'IRN',
 'Iraq': 'IRQ',
 'Ireland': 'IRL',
 'Isle of Man': 'IMN',
 'Israel': 'ISR',
 'Italy': 'ITA',
 'Jamaica': 'JAM',
 'Japan': 'JPN',
 'Jersey': 'JEY',
 'Jordan': 'JOR',
 'Kazakhstan': 'KAZ',
 'Kenya': 'KEN',
 'Kiribati': 'KIR',
 'Korea, North': 'PRK',
 'Korea, South': 'KOR',
 'Kosovo': 'KSV',
 'Kuwait': 'KWT',
 'Kyrgyzstan': 'KGZ',
 'Laos': 'LAO',
 'Latvia': 'LVA',
 'Lebanon': 'LBN',
 'Lesotho': 'LSO',
 'Liberia': 'LBR',
 'Libya': 'LBY',
 'Liechtenstein': 'LIE',
 'Lithuania': 'LTU',
 'Luxembourg': 'LUX',
 'Macau': 'MAC',
 'Macedonia': 'MKD',
 'Madagascar': 'MDG',
 'Malawi': 'MWI',
 'Malaysia': 'MYS',
 'Maldives': 'MDV',
 'Mali': 'MLI',
 'Malta': 'MLT',
 'Marshall Islands': 'MHL',
 'Mauritania': 'MRT',
 'Mauritius': 'MUS',
 'Mexico': 'MEX',
 'Micronesia, Federated States of': 'FSM',
 'Moldova': 'MDA',
 'Monaco': 'MCO',
 'Mongolia': 'MNG',
 'Montenegro': 'MNE',
 'Morocco': 'MAR',
 'Mozambique': 'MOZ',
 'Namibia': 'NAM',
 'Nepal': 'NPL',
 'Netherlands': 'NLD',
 'New Caledonia': 'NCL',
 'New Zealand': 'NZL',
 'Nicaragua': 'NIC',
 'Nigeria': 'NGA',
 'Niger': 'NER',
 'Niue': 'NIU',
 'Northern Mariana Islands': 'MNP',
 'Norway': 'NOR',
 'Oman': 'OMN',
 'Pakistan': 'PAK',
 'Palau': 'PLW',
 'Panama': 'PAN',
 'Papua New Guinea': 'PNG',
 'Paraguay': 'PRY',
 'Peru': 'PER',
 'Philippines': 'PHL',
 'Poland': 'POL',
 'Portugal': 'PRT',
 'Puerto Rico': 'PRI',
 'Qatar': 'QAT',
 'Romania': 'ROU',
 'Russia': 'RUS',
 'Rwanda': 'RWA',
 'Saint Kitts and Nevis': 'KNA',
 'Saint Lucia': 'LCA',
 'Saint Martin': 'MAF',
 'Saint Pierre and Miquelon': 'SPM',
 'Saint Vincent and the Grenadines': 'VCT',
 'Samoa': 'WSM',
 'San Marino': 'SMR',
 'Sao Tome and Principe': 'STP',
 'Saudi Arabia': 'SAU',
 'Senegal': 'SEN',
 'Serbia': 'SRB',
 'Seychelles': 'SYC',
 'Sierra Leone': 'SLE',
 'Singapore': 'SGP',
 'Sint Maarten': 'SXM',
 'Slovakia': 'SVK',
 'Slovenia': 'SVN',
 'Solomon Islands': 'SLB',
 'Somalia': 'SOM',
 'South Africa': 'ZAF',
 'South Sudan': 'SSD',
 'Spain': 'ESP',
 'Sri Lanka': 'LKA',
 'Sudan': 'SDN',
 'Suriname': 'SUR',
 'Swaziland': 'SWZ',
 'Sweden': 'SWE',
 'Switzerland': 'CHE',
 'Syria': 'SYR',
 'Taiwan': 'TWN',
 'Tajikistan': 'TJK',
 'Tanzania': 'TZA',
 'Thailand': 'THA',
 'Timor-Leste': 'TLS',
 'Togo': 'TGO',
 'Tonga': 'TON',
 'Trinidad and Tobago': 'TTO',
 'Tunisia': 'TUN',
 'Turkey': 'TUR',
 'Turkmenistan': 'TKM',
 'Tuvalu': 'TUV',
 'Uganda': 'UGA',
 'Ukraine': 'UKR',
 'United Arab Emirates': 'ARE',
    'UAE': 'ARE',
 'United Kingdom': 'GBR',
    'UK': 'GBR',
 'United States': 'USA',
    'USA': 'USA',
 'Uruguay': 'URY',
 'Uzbekistan': 'UZB',
 'Vanuatu': 'VUT',
 'Venezuela': 'VEN',
 'Vietnam': 'VNM',
 'Virgin Islands': 'VGB',
 'West Bank': 'WBG',
 'Yemen': 'YEM',
 'Zambia': 'ZMB',
 'Zimbabwe': 'ZWE',
    'S. Korea': 'KOR'
}

In [41]:
country_df['iso_code'] = country_df['Country'].map(country_abbrev)
map_country_df = country_df.loc[~country_df['iso_code'].isna()]
map_country_df['death_rate'] = (100*map_country_df['Total Deaths']/map_country_df['Total Cases']).round(2)
map_country_df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Tot Cases/1M,Deaths/1M,Total Tests,Tests/1M,Population,Continent,%Inc Cases,%Inc Deaths,%Inc Recovered,iso_code,death_rate
1,USA,108280870,9389.0,1174558,22.0,106067796,17446.0,1038516,1350,323414.0,3508.0,1186093362,3542636,334805269,North America,0.008671,0.001873,0.016448,USA,1.08
2,India,44997537,-1.0,531930,-1.0,44465019,-1.0,588,0,31990.0,378.0,930797975,661721,1406631776,Asia,-0.000002,-0.000188,-0.000002,IND,1.18
3,Germany,38428685,-1.0,174352,-1.0,38240600,-1.0,13733,0,458119.0,2078.0,122332384,1458359,83883596,Europe,-0.000003,-0.000574,-0.000003,DEU,0.45
4,Brazil,37783855,-1.0,705313,-1.0,36249161,-1.0,829381,0,175450.0,3275.0,63776166,296146,215353593,South America,-0.000003,-0.000142,-0.000003,BRA,1.87
5,S. Korea,34571873,-1.0,35934,-1.0,34129296,38989.0,406643,231,673523.0,700.0,15804065,307892,51329899,Asia,-0.000003,-0.002783,0.114239,KOR,0.10


In [42]:
fig = go.Figure(data=go.Choropleth(
    locations=map_country_df['iso_code'],
    z=map_country_df['Total Cases'].astype(float),
    colorscale="Reds",
    colorbar_title="Total"
))

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    title={'text': f"Total COVID Cases in the World", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

In [43]:
fig = go.Figure(data=go.Choropleth(
    locations=map_country_df['iso_code'],
    z=map_country_df['Total Deaths'].astype(float),
    colorscale="Reds",
    colorbar_title="Total"
))

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    title={'text': f"Total COVID Deaths in the World", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

In [44]:
fig = go.Figure(data=go.Choropleth(
    locations=map_country_df['iso_code'],
    z=map_country_df['death_rate'].astype(float),
    colorscale="Reds",
    colorbar_title="Percentage"
))

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    title={'text': f"Death Rate of Different Countries in the World", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

In [45]:
fig = go.Figure(data=go.Choropleth(
    locations=map_country_df['iso_code'],
    z=map_country_df['Deaths/1M'].astype(float),
    colorscale="Reds",
    colorbar_title="Per Million"
))

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    title={'text': f"Deaths Per Million in the World", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()
